In [1]:
# Install
!pip install -q -U google-generativeai

In [3]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

google_api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=google_api_key)

In [4]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
#Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = (
    "Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo: \n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa"
)

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="retrieval_document")

print(embeddings)

In [14]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema [...]"
}
DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entrete [...]"
}
DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada [...]"
}
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

In [25]:
model = 'models/embedding-001'

In [24]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="retrieval_document")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

In [44]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="retrieval_query")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [ ]:
consulta = "Como se troca a marcha do Googlecar?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

In [47]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
}

In [ ]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)